# Advanced Diagnostics for Confocal Microscopy Copilot
## Physics-Based Analysis for Multi-Domain Particle Tracking

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Abhishek-Gupta-GitHub/confocal_microscopy-copilot/blob/main/notebooks/PhysicsAnalyst_Advanced_Diagnostics.ipynb)

**Author:** Confocal Microscopy Copilot Team  
**Date:** December 2025  
**Version:** 1.0-hackathon

### Scope
This notebook extends the `PhysicsAnalyst` class with four advanced diagnostic modules:

1. **Near-Wall MSD** – Detect hindered diffusion near surfaces (colloids, cells on substrate)
2. **Cage-Relative MSD** – Identify local rearrangements and dynamic heterogeneity
3. **Heterogeneity Metrics** – Quantify non-Gaussian dynamics and subdiffusion anomalies
4. **Structure Functions** – Compute static structure factor and pair correlation functions

**Domain Coverage:**
- Colloidal suspensions (hard spheres, viscoelastic media, gels)
- Biological systems (nuclear diffusion, membrane proteins, vesicles)
- Soft matter (polymers, driven systems, active matter)
- Near-wall phenomena (wall-induced hindrance, confinement effects)

---

## Quick Start

If running in **Colab**, uncomment and run the setup cell below.

In [ ]:
# ============================================================================
# COLAB SETUP (uncomment if running in Google Colab)
# ============================================================================

# !git clone https://github.com/Abhishek-Gupta-GitHub/confocal_microscopy-copilot.git
# %cd confocal_microscopy-copilot
# !pip install -q -r requirements.txt
# import sys
# sys.path.insert(0, '/content/confocal_microscopy-copilot')

## Section 1: Imports and Utility Functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, interpolate, spatial, fftpack
from scipy.optimize import curve_fit
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Optional, Union
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting defaults
sns.set_theme(style='whitegrid', palette='husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ All imports successful")

## Section 2: Data Structures

Define domain-agnostic trajectory and physics dataclasses that will be exchanged with the LLM.

In [ ]:
@dataclass
class Trajectory:
    """Universal trajectory representation."""
    particle_id: int
    t: np.ndarray  # frame indices (or times)
    x: np.ndarray  # position in um (or pixels)
    y: np.ndarray
    z: np.ndarray
    confidence: Optional[np.ndarray] = None  # detection confidence [0,1] per frame
    radius: float = 0.5  # particle radius in um
    label: Optional[str] = None  # e.g., "bead", "nucleus", "vesicle"
    
    def length(self):
        """Number of points in trajectory."""
        return len(self.t)
    
    def duration(self):
        """Total duration in frames or time units."""
        return self.t[-1] - self.t[0]
    
    def positions_3d(self):
        """Return (N, 3) array of positions."""
        return np.column_stack([self.x, self.y, self.z])


@dataclass
class MSDAnalysis:
    """Mean square displacement metrics."""
    lag_times: List[float]  # tau values
    msd_bulk: List[float]  # overall MSD
    msd_wall: Optional[List[float]] = None  # near-wall MSD
    msd_cage: Optional[List[float]] = None  # cage-relative MSD
    diffusion_coeff: Optional[float] = None  # D from linear fit
    anomaly_exponent: Optional[float] = None  # alpha: MSD ~ t^alpha
    
    def to_dict(self):
        return asdict(self)


@dataclass
class HeterogeneityMetrics:
    """Non-Gaussian and heterogeneity descriptors."""
    alpha2_max: Optional[float] = None  # peak non-Gaussian parameter
    alpha2_tau_peak: Optional[float] = None  # lag time at peak alpha2
    msd_distribution_std: Optional[float] = None  # std of MSD across particles
    msd_distribution_mean: Optional[float] = None  # mean MSD
    subdiffusion_detected: bool = False  # true if alpha < 1
    superdiffusion_detected: bool = False  # true if alpha > 1
    dynamic_susceptibility: Optional[float] = None  # variance-based measure
    
    def to_dict(self):
        return asdict(self)


@dataclass
class StructureAnalysis:
    """Static structure and correlation functions."""
    q_values: List[float]  # wave vectors
    structure_factor: List[float]  # S(q) static structure factor
    pair_correlation: Optional[List[float]] = None  # g(r) radial distribution
    distances: Optional[List[float]] = None  # r values for g(r)
    density: Optional[float] = None  # particle density
    clustering_detected: bool = False  # true if S(0) >> 1
    
    def to_dict(self):
        return asdict(self)


print("✓ Data structures defined")

## Section 3: Core Diagnostic Module – PhysicsAnalyst

Comprehensive implementation of all four diagnostic categories.

In [ ]:
class PhysicsAnalyst:
    """
    Multi-domain physics analyzer for confocal microscopy trajectories.
    Computes MSD variants, heterogeneity metrics, and structure functions.
    """
    
    def __init__(self, 
                 dt: float = 1.0,  # time per frame (ms, s, etc.)
                 px_to_um: float = 1.0,  # pixel to micrometer conversion
                 wall_z_threshold: float = 2.0):  # wall proximity threshold (um)
        """
        Parameters
        ----------
        dt : float
            Time interval per frame (units: ms, s, etc.). Used for physical time axes.
        px_to_um : float
            Pixel-to-micrometer conversion factor.
        wall_z_threshold : float
            Distance threshold below which particles are considered "near-wall".
        """
        self.dt = dt
        self.px_to_um = px_to_um
        self.wall_z_threshold = wall_z_threshold
    
    # ========================================================================
    # CORE MSD COMPUTATION
    # ========================================================================
    
    def compute_msd(self, trajectory: Trajectory, 
                   max_lag_frames: Optional[int] = None,
                   stride: int = 1) -> Tuple[np.ndarray, np.ndarray]:
        """
        Compute mean square displacement.
        
        Parameters
        ----------
        trajectory : Trajectory
            Single particle trajectory.
        max_lag_frames : int, optional
            Maximum lag time in frames. Default: 1/4 of trajectory length.
        stride : int
            Compute MSD at every stride-th lag time (reduce computation).
        
        Returns
        -------
        lag_times : ndarray
            Lag times in physical units (dt * frames).
        msd : ndarray
            Mean square displacement in (um)^2.
        """
        pos = trajectory.positions_3d() * self.px_to_um
        N = len(pos)
        
        if max_lag_frames is None:
            max_lag_frames = max(N // 4, 1)
        
        max_lag_frames = min(max_lag_frames, N - 1)
        lag_frames = np.arange(0, max_lag_frames + 1, stride)
        msd = np.zeros(len(lag_frames))
        
        for i, tau_frame in enumerate(lag_frames):
            if tau_frame == 0:
                msd[i] = 0.0
            else:
                disp = pos[tau_frame:] - pos[:-tau_frame]
                msd[i] = np.mean(np.sum(disp**2, axis=1))
        
        lag_times = lag_frames * self.dt
        return lag_times, msd
    
    # ========================================================================
    # 1. NEAR-WALL MSD
    # ========================================================================
    
    def compute_wall_proximity_mask(self, trajectory: Trajectory) -> np.ndarray:
        """
        Identify frames where particle is near the wall (bottom substrate).
        
        Returns
        -------
        mask : ndarray of bool
            True where z < wall_z_threshold.
        """
        z_scaled = trajectory.z * self.px_to_um
        return z_scaled < self.wall_z_threshold
    
    def compute_near_wall_msd(self, trajectory: Trajectory,
                              max_lag_frames: Optional[int] = None,
                              stride: int = 1) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Compute MSD for trajectories near the wall (z < threshold).
        
        Compare with bulk MSD to detect wall-induced hindrance.
        
        Returns
        -------
        lag_times : ndarray
            Lag times in physical units.
        msd_wall : ndarray
            MSD for near-wall segments.
        msd_bulk : ndarray
            MSD for bulk (far-wall) segments.
        """
        pos = trajectory.positions_3d() * self.px_to_um
        wall_mask = self.compute_wall_proximity_mask(trajectory)
        
        N = len(pos)
        if max_lag_frames is None:
            max_lag_frames = max(N // 4, 1)
        
        max_lag_frames = min(max_lag_frames, N - 1)
        lag_frames = np.arange(0, max_lag_frames + 1, stride)
        
        msd_wall = np.zeros(len(lag_frames))
        msd_bulk = np.zeros(len(lag_frames))
        counts_wall = np.zeros(len(lag_frames))
        counts_bulk = np.zeros(len(lag_frames))
        
        for i, tau_frame in enumerate(lag_frames):
            if tau_frame == 0:
                msd_wall[i] = 0.0
                msd_bulk[i] = 0.0
            else:
                disp = pos[tau_frame:] - pos[:-tau_frame]
                displacements_sq = np.sum(disp**2, axis=1)
                
                # Condition on wall proximity at start of interval
                wall_starts = wall_mask[:-tau_frame]
                bulk_starts = ~wall_mask[:-tau_frame]
                
                if wall_starts.sum() > 0:
                    msd_wall[i] = np.mean(displacements_sq[wall_starts])
                    counts_wall[i] = wall_starts.sum()
                
                if bulk_starts.sum() > 0:
                    msd_bulk[i] = np.mean(displacements_sq[bulk_starts])
                    counts_bulk[i] = bulk_starts.sum()
        
        lag_times = lag_frames * self.dt
        return lag_times, msd_wall, msd_bulk
    
    # ========================================================================
    # 2. CAGE-RELATIVE MSD
    # ========================================================================
    
    def compute_cage_relative_msd(self, trajectories: List[Trajectory],
                                  reference_traj_idx: int,
                                  cage_radius: float = 5.0,  # um
                                  max_lag_frames: Optional[int] = None,
                                  stride: int = 1) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Compute cage-relative MSD for a single particle.
        
        This subtracts the center-of-mass motion of the local neighborhood,
        revealing rearrangements vs. affine (bulk) motion.
        
        Parameters
        ----------
        trajectories : List[Trajectory]
            All trajectories in the system.
        reference_traj_idx : int
            Index of particle for which to compute cage-relative MSD.
        cage_radius : float
            Neighborhood radius for defining the cage (um).
        max_lag_frames : int, optional
            Maximum lag time in frames.
        stride : int
            Stride for lag times.
        
        Returns
        -------
        lag_times : ndarray
            Lag times.
        msd_cage : ndarray
            Cage-relative MSD.
        msd_bulk : ndarray
            Bulk (non-relative) MSD for reference.
        """
        ref_traj = trajectories[reference_traj_idx]
        ref_pos = ref_traj.positions_3d() * self.px_to_um
        N = len(ref_pos)
        
        if max_lag_frames is None:
            max_lag_frames = max(N // 4, 1)
        
        # Identify cage neighbors (within cage_radius at t=0)
        t0_ref_pos = ref_pos[0]
        cage_neighbors = []
        
        for j, traj in enumerate(trajectories):
            if j == reference_traj_idx:
                continue
            other_pos = traj.positions_3d() * self.px_to_um
            dist_to_ref = np.linalg.norm(other_pos[0] - t0_ref_pos)
            if dist_to_ref < cage_radius:
                cage_neighbors.append(j)
        
        if not cage_neighbors:
            # No neighbors; fall back to bulk MSD
            lag_times, msd_bulk = self.compute_msd(ref_traj, max_lag_frames, stride)
            return lag_times, msd_bulk.copy(), msd_bulk.copy()
        
        # Build neighbor position matrix: (N_frames, N_neighbors, 3)
        neighbor_positions = np.stack(
            [trajectories[j].positions_3d() * self.px_to_um for j in cage_neighbors],
            axis=1
        )
        
        # Compute COM of cage at each time step
        cage_com = np.mean(neighbor_positions, axis=1)  # (N_frames, 3)
        
        # Compute cage-relative position (reference particle - cage COM)
        rel_pos = ref_pos - cage_com
        
        max_lag_frames = min(max_lag_frames, N - 1)
        lag_frames = np.arange(0, max_lag_frames + 1, stride)
        
        msd_cage = np.zeros(len(lag_frames))
        msd_bulk = np.zeros(len(lag_frames))
        
        for i, tau_frame in enumerate(lag_frames):
            if tau_frame == 0:
                msd_cage[i] = 0.0
                msd_bulk[i] = 0.0
            else:
                # Bulk MSD
                disp_bulk = ref_pos[tau_frame:] - ref_pos[:-tau_frame]
                msd_bulk[i] = np.mean(np.sum(disp_bulk**2, axis=1))
                
                # Cage-relative MSD
                disp_rel = rel_pos[tau_frame:] - rel_pos[:-tau_frame]
                msd_cage[i] = np.mean(np.sum(disp_rel**2, axis=1))
        
        lag_times = lag_frames * self.dt
        return lag_times, msd_cage, msd_bulk
    
    # ========================================================================
    # 3. HETEROGENEITY METRICS
    # ========================================================================
    
    def fit_msd_power_law(self, lag_times: np.ndarray, msd: np.ndarray,
                          window_fraction: float = 0.3) -> Tuple[float, float]:
        """
        Fit MSD ~ t^alpha in linear region to extract anomaly exponent.
        
        Parameters
        ----------
        lag_times : ndarray
            Lag times.
        msd : ndarray
            Mean square displacements.
        window_fraction : float
            Use this fraction of data in linear regime (default: first 30%).
        
        Returns
        -------
        alpha : float
            Anomaly exponent (1=Brownian, <1=subdiffusion, >1=superdiffusion).
        D_alpha : float
            Generalized diffusion coefficient.
        """
        # Use early-time window to avoid noise and saturation
        n_points = max(int(len(lag_times) * window_fraction), 2)
        
        # Skip lag=0
        idx = slice(1, n_points + 1)
        
        log_tau = np.log(lag_times[idx])
        log_msd = np.log(np.maximum(msd[idx], 1e-10))
        
        # Linear fit in log-log space
        coeffs = np.polyfit(log_tau, log_msd, 1)
        alpha = coeffs[0]
        D_alpha = np.exp(coeffs[1]) / (2 * 3)  # generalized D (3D)
        
        return alpha, D_alpha
    
    def compute_non_gaussian_parameter(self, trajectories: List[Trajectory],
                                       max_lag_frames: Optional[int] = None,
                                       stride: int = 1) -> Tuple[np.ndarray, np.ndarray]:
        """
        Compute non-Gaussian parameter α2(τ) across particle ensemble.
        
        α2(τ) = <r^4(τ)> / (3 * <r^2(τ)>^2) - 1
        
        α2 ≈ 0 for Gaussian displacements, α2 > 0 indicates non-Gaussian tails.
        
        Parameters
        ----------
        trajectories : List[Trajectory]
            All trajectories.
        max_lag_frames : int, optional
            Maximum lag time.
        stride : int
            Stride in lag times.
        
        Returns
        -------
        lag_times : ndarray
            Lag times.
        alpha2 : ndarray
            Non-Gaussian parameter vs lag time.
        """
        if not trajectories:
            return np.array([0.0]), np.array([0.0])
        
        N_frames = max([len(t.t) for t in trajectories])
        if max_lag_frames is None:
            max_lag_frames = max(N_frames // 4, 1)
        
        max_lag_frames = min(max_lag_frames, N_frames - 1)
        lag_frames = np.arange(0, max_lag_frames + 1, stride)
        alpha2 = np.zeros(len(lag_frames))
        
        for i, tau_frame in enumerate(lag_frames):
            if tau_frame == 0:
                alpha2[i] = 0.0
                continue
            
            r2_list = []
            r4_list = []
            
            for traj in trajectories:
                if len(traj.t) <= tau_frame:
                    continue
                
                pos = traj.positions_3d() * self.px_to_um
                disp = pos[tau_frame:] - pos[:-tau_frame]
                r2 = np.sum(disp**2, axis=1)
                r2_list.extend(r2)
                r4_list.extend(r2**2)
            
            if len(r2_list) > 1:
                r2_mean = np.mean(r2_list)
                r4_mean = np.mean(r4_list)
                alpha2[i] = (r4_mean / (3 * r2_mean**2)) - 1.0 if r2_mean > 0 else 0.0
            else:
                alpha2[i] = 0.0
        
        lag_times = lag_frames * self.dt
        return lag_times, alpha2
    
    def compute_heterogeneity_metrics(self, trajectories: List[Trajectory],
                                      max_lag_frames: Optional[int] = None) -> HeterogeneityMetrics:
        """
        Compute full heterogeneity analysis across trajectory ensemble.
        
        Returns
        -------
        HeterogeneityMetrics
            Dataclass with all heterogeneity descriptors.
        """
        if not trajectories:
            return HeterogeneityMetrics()
        
        # Compute per-particle anomaly exponents
        alphas = []
        msd_at_long_tau = []
        
        for traj in trajectories:
            lag_times, msd = self.compute_msd(traj, max_lag_frames)
            if len(lag_times) > 1:
                alpha, _ = self.fit_msd_power_law(lag_times, msd, window_fraction=0.3)
                alphas.append(alpha)
                msd_at_long_tau.append(msd[-1])  # MSD at longest lag
        
        # Non-Gaussian parameter
        lag_times, alpha2_array = self.compute_non_gaussian_parameter(trajectories, max_lag_frames)
        alpha2_max = float(np.max(alpha2_array)) if len(alpha2_array) > 0 else None
        alpha2_tau_peak = float(lag_times[np.argmax(alpha2_array)]) if len(alpha2_array) > 0 else None
        
        # Distribution statistics
        msd_mean = float(np.mean(msd_at_long_tau)) if msd_at_long_tau else None
        msd_std = float(np.std(msd_at_long_tau)) if msd_at_long_tau else None
        
        # Subdiffusion/superdiffusion detection
        mean_alpha = float(np.mean(alphas)) if alphas else 1.0
        subdiffusion = mean_alpha < 0.9
        superdiffusion = mean_alpha > 1.1
        
        # Dynamic susceptibility (as proxy for heterogeneity)
        dynamic_susceptibility = msd_std / msd_mean if (msd_mean and msd_std) else None
        
        return HeterogeneityMetrics(
            alpha2_max=alpha2_max,
            alpha2_tau_peak=alpha2_tau_peak,
            msd_distribution_std=msd_std,
            msd_distribution_mean=msd_mean,
            subdiffusion_detected=subdiffusion,
            superdiffusion_detected=superdiffusion,
            dynamic_susceptibility=dynamic_susceptibility
        )
    
    # ========================================================================
    # 4. STRUCTURE FUNCTIONS
    # ========================================================================
    
    def compute_static_structure_factor(self, trajectories: List[Trajectory],
                                       time_window_fraction: float = 0.5,
                                       n_q_bins: int = 20) -> Tuple[np.ndarray, np.ndarray]:
        """
        Compute static structure factor S(q) from spatial positions.
        
        S(q) = 1 + (1/N) * |sum_j exp(i q · r_j)|^2
        
        Parameters
        ----------
        trajectories : List[Trajectory]
            All trajectories.
        time_window_fraction : float
            Use last fraction of trajectory for averaging.
        n_q_bins : int
            Number of q-vector magnitudes.
        
        Returns
        -------
        q_values : ndarray
            Wave vector magnitudes (1/um).
        S_q : ndarray
            Structure factor S(q).
        """
        if not trajectories:
            return np.array([0.0]), np.array([0.0])
        
        # Collect positions from late-time window
        all_positions = []
        for traj in trajectories:
            pos = traj.positions_3d() * self.px_to_um
            t_start = int(len(pos) * (1 - time_window_fraction))
            all_positions.append(np.mean(pos[t_start:], axis=0))  # time-averaged position
        
        positions = np.array(all_positions)  # (N_particles, 3)
        N = len(positions)
        
        if N < 2:
            return np.array([0.0]), np.array([1.0])
        
        # Define q-space grid
        # Typical q range: 2*pi / L where L ~ system size
        system_size = np.max(np.ptp(positions, axis=0))
        max_q = 2 * np.pi / (system_size / 10)  # sample ~10 wavelengths
        q_values = np.linspace(0.1, max_q, n_q_bins)
        
        S_q = np.ones_like(q_values)
        
        for i, q_mag in enumerate(q_values):
            # Random q-vector direction for isotropic average
            theta = np.random.rand() * 2 * np.pi
            phi = np.random.rand() * np.pi
            q_vec = q_mag * np.array([np.sin(phi) * np.cos(theta),
                                       np.sin(phi) * np.sin(theta),
                                       np.cos(phi)])
            
            # Compute |sum exp(i q·r)|^2
            phase = np.dot(positions, q_vec)
            sum_exp = np.sum(np.exp(1j * phase))
            S_q[i] = 1.0 + (np.abs(sum_exp)**2 / N)
        
        return q_values, S_q
    
    def compute_pair_correlation(self, trajectories: List[Trajectory],
                                time_window_fraction: float = 0.5,
                                n_r_bins: int = 30) -> Tuple[np.ndarray, np.ndarray]:
        """
        Compute radial distribution function g(r).
        
        Parameters
        ----------
        trajectories : List[Trajectory]
            All trajectories.
        time_window_fraction : float
            Use last fraction of trajectory for averaging.
        n_r_bins : int
            Number of radial bins.
        
        Returns
        -------
        r_values : ndarray
            Distances (um).
        g_r : ndarray
            Radial distribution function.
        """
        if not trajectories:
            return np.array([0.0]), np.array([0.0])
        
        # Collect positions from late-time window
        all_positions = []
        for traj in trajectories:
            pos = traj.positions_3d() * self.px_to_um
            t_start = int(len(pos) * (1 - time_window_fraction))
            all_positions.append(np.mean(pos[t_start:], axis=0))
        
        positions = np.array(all_positions)
        N = len(positions)
        
        if N < 2:
            return np.array([0.0]), np.array([0.0])
        
        # Compute pairwise distances
        distances = spatial.distance.pdist(positions, metric='euclidean')
        
        # Bin distances
        r_max = np.max(distances)
        r_bins = np.linspace(0, r_max, n_r_bins)
        g_r, _ = np.histogram(distances, bins=r_bins, density=False)
        
        # Normalize by bin volume and density
        dr = r_bins[1] - r_bins[0] if len(r_bins) > 1 else 1.0
        r_centers = (r_bins[:-1] + r_bins[1:]) / 2
        bin_volume = 4 * np.pi * r_centers**2 * dr
        
        # Expected count for random distribution
        density = N / (np.ptp(positions, axis=0).prod() if np.all(np.ptp(positions, axis=0) > 0) else 1.0)
        expected_count = bin_volume * density * (N - 1) / 2  # pairs
        
        # Avoid division by zero
        g_r_normalized = np.where(expected_count > 0, g_r / expected_count, 0.0)
        
        return r_centers, g_r_normalized
    
    def compute_structure_analysis(self, trajectories: List[Trajectory]) -> StructureAnalysis:
        """
        Compute full structure analysis (S(q) and g(r)).
        
        Returns
        -------
        StructureAnalysis
            Dataclass with structure factor and pair correlation.
        """
        q_vals, S_q = self.compute_static_structure_factor(trajectories)
        r_vals, g_r = self.compute_pair_correlation(trajectories)
        
        density = len(trajectories) / (np.mean([np.ptp(t.positions_3d(), axis=0).prod() for t in trajectories]) * (self.px_to_um**3) + 1e-10)
        
        clustering = S_q[0] > 1.5 if len(S_q) > 0 else False  # S(0) >> 1 indicates clustering
        
        return StructureAnalysis(
            q_values=q_vals.tolist(),
            structure_factor=S_q.tolist(),
            pair_correlation=g_r.tolist(),
            distances=r_vals.tolist(),
            density=float(density),
            clustering_detected=clustering
        )
    
    # ========================================================================
    # AGGREGATED SUMMARY (JSON for LLM)
    # ========================================================================
    
    def summarize_physics(self, trajectories: List[Trajectory],
                          domain: str = "general") -> Dict:
        """
        Generate comprehensive physics summary as JSON for LLM + UI.
        
        Parameters
        ----------
        trajectories : List[Trajectory]
            All particle trajectories.
        domain : str
            Domain context: "colloidal", "cellular", "polymer", or "general".
        
        Returns
        -------
        dict
            Comprehensive JSON summary with all metrics.
        """
        if not trajectories:
            return {"error": "No trajectories provided"}
        
        # 1. Bulk MSD
        ref_traj = trajectories[0]
        lag_times, msd_bulk = self.compute_msd(ref_traj)
        alpha_bulk, D_bulk = self.fit_msd_power_law(lag_times, msd_bulk)
        
        # 2. Near-wall MSD (if applicable)
        lag_times_wall, msd_wall, msd_bulk_full = self.compute_near_wall_msd(ref_traj)
        alpha_wall, D_wall = self.fit_msd_power_law(lag_times_wall, np.maximum(msd_wall, 1e-10))
        
        # 3. Cage-relative MSD (if multiple particles)
        if len(trajectories) > 1:
            lag_times_cage, msd_cage, msd_bulk_cage = self.compute_cage_relative_msd(
                trajectories, reference_traj_idx=0
            )
            alpha_cage, D_cage = self.fit_msd_power_law(lag_times_cage, msd_cage)
        else:
            lag_times_cage, msd_cage, msd_bulk_cage = lag_times, msd_bulk.copy(), msd_bulk.copy()
            alpha_cage, D_cage = alpha_bulk, D_bulk
        
        # 4. Heterogeneity
        hetero = self.compute_heterogeneity_metrics(trajectories)
        
        # 5. Structure
        struct = self.compute_structure_analysis(trajectories)
        
        # Compile JSON
        summary = {
            "metadata": {
                "domain": domain,
                "n_trajectories": len(trajectories),
                "mean_trajectory_length": float(np.mean([len(t.t) for t in trajectories])),
                "dt_physical_units": self.dt,
                "px_to_um_conversion": self.px_to_um,
            },
            "msd_analysis": {
                "bulk": {
                    "lag_times": lag_times.tolist(),
                    "msd_values": msd_bulk.tolist(),
                    "anomaly_exponent_alpha": float(alpha_bulk),
                    "diffusion_coefficient": float(D_bulk),
                    "interpretation": self._interpret_msd_exponent(alpha_bulk)
                },
                "near_wall": {
                    "lag_times": lag_times_wall.tolist(),
                    "msd_wall": msd_wall.tolist(),
                    "msd_bulk": msd_bulk_full.tolist(),
                    "wall_hindrance_ratio": float(np.mean(msd_bulk_full[1:] / (msd_wall[1:] + 1e-10))) if len(msd_wall) > 1 else 1.0,
                    "anomaly_exponent_wall": float(alpha_wall),
                },
                "cage_relative": {
                    "lag_times": lag_times_cage.tolist(),
                    "msd_cage": msd_cage.tolist(),
                    "msd_reference": msd_bulk_cage.tolist(),
                    "anomaly_exponent_cage": float(alpha_cage),
                }
            },
            "heterogeneity": asdict(hetero),
            "structure": asdict(struct),
            "flags_and_anomalies": self._generate_flags(alpha_bulk, hetero, struct, domain)
        }
        
        return summary
    
    def _interpret_msd_exponent(self, alpha: float) -> str:
        """Interpret MSD exponent value."""
        if alpha < 0.8:
            return "Strong subdiffusion (possibly caged/glassy)"
        elif alpha < 1.0:
            return "Subdiffusion (viscoelastic or confined)"
        elif 0.95 < alpha < 1.05:
            return "Brownian diffusion (normal)"
        elif alpha <= 1.2:
            return "Slight superdiffusion (transient dynamics)"
        else:
            return "Strong superdiffusion (active or ballistic)"
    
    def _generate_flags(self, alpha: float, hetero: HeterogeneityMetrics, 
                       struct: StructureAnalysis, domain: str) -> List[str]:
        """Generate domain-aware flags and anomalies."""
        flags = []
        
        # MSD-based flags
        if hetero.subdiffusion_detected:
            flags.append(f"subdiffusion_detected: alpha < 0.9 (viscoelasticity or confinement)")
        if hetero.superdiffusion_detected:
            flags.append(f"superdiffusion_detected: alpha > 1.1 (active transport or drift)")
        
        # Heterogeneity flags
        if hetero.alpha2_max is not None and hetero.alpha2_max > 0.1:
            flags.append(f"non_gaussian_dynamics: alpha2_max = {hetero.alpha2_max:.3f} at tau = {hetero.alpha2_tau_peak}")
        
        if hetero.dynamic_susceptibility is not None and hetero.dynamic_susceptibility > 0.3:
            flags.append(f"strong_heterogeneity: MSD variance/mean ratio = {hetero.dynamic_susceptibility:.2f}")
        
        # Structure flags
        if struct.clustering_detected:
            flags.append(f"spatial_clustering: S(0) = {struct.structure_factor[0]:.2f} (non-random distribution)")
        
        # Domain-specific flags
        if domain == "colloidal" and hetero.subdiffusion_detected:
            flags.append("colloidal_context: subdiffusion may indicate gel formation or jamming")
        elif domain == "cellular" and struct.clustering_detected:
            flags.append("cellular_context: clustering may indicate organelle co-localization")
        elif domain == "polymer" and hetero.alpha2_max is not None and hetero.alpha2_max > 0.2:
            flags.append("polymer_context: non-Gaussian dynamics consistent with polymer chain dynamics")
        
        return flags if flags else ["system_behaves_normally_no_anomalies_detected"]


print("✓ PhysicsAnalyst class defined successfully")

## Section 4: Demonstration on Synthetic Data

Generate realistic synthetic trajectories and run full diagnostics.

In [ ]:
# ============================================================================
# SYNTHETIC DATA GENERATION
# ============================================================================

def generate_brownian_trajectory(n_frames: int, dt: float = 0.1, 
                                 diffusion_coeff: float = 1.0,
                                 start_pos: tuple = (0, 0, 0)) -> Trajectory:
    """
    Generate Brownian motion trajectory.
    
    MSD(t) = 6 * D * t (for 3D)
    """
    sigma = np.sqrt(2 * diffusion_coeff * dt)
    steps = np.random.normal(0, sigma, (n_frames, 3))
    pos = np.cumsum(steps, axis=0) + np.array(start_pos)
    
    return Trajectory(
        particle_id=0,
        t=np.arange(n_frames),
        x=pos[:, 0],
        y=pos[:, 1],
        z=pos[:, 2],
        radius=0.5,
        label="bead"
    )


def generate_subdiffusive_trajectory(n_frames: int, dt: float = 0.1,
                                      alpha: float = 0.7, start_pos: tuple = (0, 0, 0)) -> Trajectory:
    """
    Generate subdiffusive (fractional Brownian motion) trajectory.
    MSD(t) ~ t^alpha with alpha < 1.
    """
    # Simplified: use correlated steps
    steps = np.random.normal(0, 1.0, (n_frames, 3))
    
    # Apply temporal correlations to reduce alpha
    for i in range(1, n_frames):
        steps[i] = (1 - alpha) * steps[i] + alpha * steps[i - 1]
    
    pos = np.cumsum(steps * np.sqrt(dt), axis=0) + np.array(start_pos)
    
    return Trajectory(
        particle_id=0,
        t=np.arange(n_frames),
        x=pos[:, 0],
        y=pos[:, 1],
        z=pos[:, 2],
        radius=0.5,
        label="confined_particle"
    )


def generate_ensemble(n_particles: int, n_frames: int, dt: float = 0.1,
                     domain: str = "colloidal") -> List[Trajectory]:
    """
    Generate an ensemble of particles with domain-specific properties.
    
    Parameters
    ----------
    domain : str
        "colloidal", "cellular", "polymer", or "mixed_heterogeneous"
    """
    trajectories = []
    
    if domain == "colloidal":
        # Hard sphere colloids: Brownian motion
        for i in range(n_particles):
            start_pos = (np.random.uniform(-10, 10), 
                         np.random.uniform(-10, 10), 
                         np.random.uniform(2, 8))
            traj = generate_brownian_trajectory(n_frames, dt=dt, diffusion_coeff=1.0, start_pos=start_pos)
            traj.particle_id = i
            traj.label = "colloidal_bead"
            trajectories.append(traj)
    
    elif domain == "cellular":
        # Cells/vesicles: constrained + occasional jumps (active transport)
        for i in range(n_particles):
            start_pos = (np.random.uniform(-5, 5),
                         np.random.uniform(-5, 5),
                         np.random.uniform(1, 4))
            traj = generate_subdiffusive_trajectory(n_frames, dt=dt, alpha=0.8, start_pos=start_pos)
            traj.particle_id = i
            traj.label = "nucleus" if i % 2 == 0 else "vesicle"
            trajectories.append(traj)
    
    elif domain == "polymer":
        # Polymers: strong subdiffusion + highly heterogeneous
        for i in range(n_particles):
            alpha = np.random.uniform(0.5, 0.8)  # variable alpha
            start_pos = (np.random.uniform(-3, 3),
                         np.random.uniform(-3, 3),
                         np.random.uniform(0.5, 3))
            traj = generate_subdiffusive_trajectory(n_frames, dt=dt, alpha=alpha, start_pos=start_pos)
            traj.particle_id = i
            traj.label = "polymer_bead"
            trajectories.append(traj)
    
    elif domain == "mixed_heterogeneous":
        # Mixture: some Brownian, some subdiffusive
        for i in range(n_particles):
            start_pos = (np.random.uniform(-8, 8),
                         np.random.uniform(-8, 8),
                         np.random.uniform(1, 6))
            if i < n_particles // 2:
                traj = generate_brownian_trajectory(n_frames, dt=dt, diffusion_coeff=1.0, start_pos=start_pos)
                traj.label = "mobile_particle"
            else:
                traj = generate_subdiffusive_trajectory(n_frames, dt=dt, alpha=0.75, start_pos=start_pos)
                traj.label = "confined_particle"
            traj.particle_id = i
            trajectories.append(traj)
    
    return trajectories


print("✓ Synthetic data generators ready")

## Section 5: Run Analysis on Synthetic Dataset

In [ ]:
# ============================================================================
# EXAMPLE 1: Colloidal System (Brownian, No Heterogeneity)
# ============================================================================

print("="*70)
print("EXAMPLE 1: Colloidal System")
print("="*70)

# Generate ensemble
trajectories_colloidal = generate_ensemble(n_particles=10, n_frames=500, dt=0.01, domain="colloidal")

print(f"Generated {len(trajectories_colloidal)} colloidal bead trajectories")
print(f"Mean trajectory length: {np.mean([len(t.t) for t in trajectories_colloidal]):.0f} frames")

# Initialize analyzer
analyzer_colloidal = PhysicsAnalyst(dt=0.01, px_to_um=0.065, wall_z_threshold=2.0)

# Run full analysis
summary_colloidal = analyzer_colloidal.summarize_physics(trajectories_colloidal, domain="colloidal")

# Print key results
print(f"\nBulk MSD Analysis:")
print(f"  - Anomaly exponent (α): {summary_colloidal['msd_analysis']['bulk']['anomaly_exponent_alpha']:.3f}")
print(f"  - Diffusion coeff: {summary_colloidal['msd_analysis']['bulk']['diffusion_coefficient']:.3e}")
print(f"  - Interpretation: {summary_colloidal['msd_analysis']['bulk']['interpretation']}")

print(f"\nHeterogeneity Metrics:")
print(f"  - α₂ max: {summary_colloidal['heterogeneity']['alpha2_max']}")
print(f"  - Subdiffusion detected: {summary_colloidal['heterogeneity']['subdiffusion_detected']}")
print(f"  - Dynamic susceptibility: {summary_colloidal['heterogeneity']['dynamic_susceptibility']}")

print(f"\nStructure Analysis:")
print(f"  - Clustering detected: {summary_colloidal['structure']['clustering_detected']}")
print(f"  - Number of q-values: {len(summary_colloidal['structure']['q_values'])}")

print(f"\nFlags/Anomalies:")
for flag in summary_colloidal['flags_and_anomalies']:
    print(f"  - {flag}")

In [ ]:
# ============================================================================
# EXAMPLE 2: Cellular System (Subdiffusion + Heterogeneity)
# ============================================================================

print("\n" + "="*70)
print("EXAMPLE 2: Cellular System")
print("="*70)

trajectories_cellular = generate_ensemble(n_particles=15, n_frames=500, dt=0.01, domain="cellular")

print(f"Generated {len(trajectories_cellular)} cellular trajectories (nuclei + vesicles)")

analyzer_cellular = PhysicsAnalyst(dt=0.01, px_to_um=0.065, wall_z_threshold=2.0)
summary_cellular = analyzer_cellular.summarize_physics(trajectories_cellular, domain="cellular")

print(f"\nBulk MSD Analysis:")
print(f"  - Anomaly exponent (α): {summary_cellular['msd_analysis']['bulk']['anomaly_exponent_alpha']:.3f}")
print(f"  - Interpretation: {summary_cellular['msd_analysis']['bulk']['interpretation']}")

print(f"\nHeterogeneity Metrics:")
print(f"  - α₂ max: {summary_cellular['heterogeneity']['alpha2_max']}")
print(f"  - Subdiffusion detected: {summary_cellular['heterogeneity']['subdiffusion_detected']}")
print(f"  - MSD distribution std: {summary_cellular['heterogeneity']['msd_distribution_std']}")

print(f"\nFlags/Anomalies:")
for flag in summary_cellular['flags_and_anomalies']:
    print(f"  - {flag}")

In [ ]:
# ============================================================================
# EXAMPLE 3: Polymer System (Strong Subdiffusion + Heterogeneity)
# ============================================================================

print("\n" + "="*70)
print("EXAMPLE 3: Polymer System")
print("="*70)

trajectories_polymer = generate_ensemble(n_particles=12, n_frames=500, dt=0.01, domain="polymer")

print(f"Generated {len(trajectories_polymer)} polymer bead trajectories")

analyzer_polymer = PhysicsAnalyst(dt=0.01, px_to_um=0.065, wall_z_threshold=2.0)
summary_polymer = analyzer_polymer.summarize_physics(trajectories_polymer, domain="polymer")

print(f"\nBulk MSD Analysis:")
print(f"  - Anomaly exponent (α): {summary_polymer['msd_analysis']['bulk']['anomaly_exponent_alpha']:.3f}")
print(f"  - Interpretation: {summary_polymer['msd_analysis']['bulk']['interpretation']}")

print(f"\nHeterogeneity Metrics:")
print(f"  - α₂ max: {summary_polymer['heterogeneity']['alpha2_max']}")
print(f"  - Dynamic susceptibility: {summary_polymer['heterogeneity']['dynamic_susceptibility']}")

print(f"\nFlags/Anomalies:")
for flag in summary_polymer['flags_and_anomalies']:
    print(f"  - {flag}")

## Section 6: Visualization and Interpretation

Plot all four diagnostic categories side-by-side.

In [ ]:
def plot_diagnostics(analyzer: PhysicsAnalyst, 
                    trajectories: List[Trajectory],
                    summary: Dict,
                    title: str = "Physics Diagnostics"):
    """
    Create comprehensive visualization of all four diagnostic modules.
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    # ====== Panel 1: Bulk + Wall + Cage MSD ======
    ax = axes[0, 0]
    
    bulk = summary['msd_analysis']['bulk']
    wall = summary['msd_analysis']['near_wall']
    cage = summary['msd_analysis']['cage_relative']
    
    ax.loglog(bulk['lag_times'], bulk['msd_values'], 'o-', label='Bulk MSD', linewidth=2, markersize=6)
    ax.loglog(wall['lag_times'], wall['msd_wall'], 's--', label='Near-wall MSD', linewidth=2, markersize=5, alpha=0.7)
    ax.loglog(cage['lag_times'], cage['msd_cage'], '^:', label='Cage-relative MSD', linewidth=2, markersize=5, alpha=0.7)
    
    # Overlay power law fit
    alpha = bulk['anomaly_exponent_alpha']
    tau_fit = np.array(bulk['lag_times'])[1:10]
    msd_fit = tau_fit ** alpha * (bulk['msd_values'][5] / (tau_fit[3]**alpha))
    ax.loglog(tau_fit, msd_fit, 'k--', alpha=0.5, label=f"α={alpha:.2f}")
    
    ax.set_xlabel('Lag time (physical units)', fontsize=11)
    ax.set_ylabel('MSD (μm²)', fontsize=11)
    ax.set_title('1. MSD Analysis: Bulk, Wall, and Cage-Relative', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, which='both', alpha=0.3)
    
    # ====== Panel 2: Non-Gaussian Parameter α₂(τ) ======
    ax = axes[0, 1]
    
    lag_times_het, alpha2_array = analyzer.compute_non_gaussian_parameter(trajectories)
    
    ax.plot(lag_times_het, alpha2_array, 'o-', color='tab:red', linewidth=2, markersize=6, label='α₂(τ)')
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.5, label='Gaussian threshold')
    ax.fill_between(lag_times_het, 0, alpha2_array, alpha=0.3, color='tab:red')
    
    hetero = summary['heterogeneity']
    if hetero['alpha2_max']:
        ax.plot(hetero['alpha2_tau_peak'], hetero['alpha2_max'], 'r*', markersize=20, 
               label=f"Peak α₂={hetero['alpha2_max']:.3f}")
    
    ax.set_xlabel('Lag time (physical units)', fontsize=11)
    ax.set_ylabel('Non-Gaussian Parameter α₂(τ)', fontsize=11)
    ax.set_title('2. Heterogeneity: Non-Gaussian Dynamics', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # ====== Panel 3: Structure Factor S(q) ======
    ax = axes[1, 0]
    
    struct = summary['structure']
    ax.plot(struct['q_values'], struct['structure_factor'], 'o-', color='tab:green', linewidth=2, markersize=6)
    ax.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Random distribution')
    ax.fill_between(struct['q_values'], 1, struct['structure_factor'], alpha=0.3, color='tab:green')
    
    ax.set_xlabel('Wave vector q (1/μm)', fontsize=11)
    ax.set_ylabel('Structure Factor S(q)', fontsize=11)
    ax.set_title('3. Structure Analysis: Static Structure Factor', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # ====== Panel 4: Pair Correlation g(r) ======
    ax = axes[1, 1]
    
    if struct['pair_correlation'] and struct['distances']:
        ax.plot(struct['distances'], struct['pair_correlation'], 'o-', color='tab:purple', linewidth=2, markersize=6)
        ax.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Random distribution')
        ax.fill_between(struct['distances'], 1, struct['pair_correlation'], alpha=0.3, color='tab:purple')
        
        ax.set_xlabel('Pairwise distance (μm)', fontsize=11)
        ax.set_ylabel('Radial Distribution Function g(r)', fontsize=11)
    else:
        ax.text(0.5, 0.5, "Insufficient data for g(r)", ha='center', va='center', fontsize=12)
    
    ax.set_title('4. Structure Analysis: Radial Distribution', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


# Plot all three examples
fig1 = plot_diagnostics(analyzer_colloidal, trajectories_colloidal, summary_colloidal, "Colloidal System Diagnostics")
plt.show()

fig2 = plot_diagnostics(analyzer_cellular, trajectories_cellular, summary_cellular, "Cellular System Diagnostics")
plt.show()

fig3 = plot_diagnostics(analyzer_polymer, trajectories_polymer, summary_polymer, "Polymer System Diagnostics")
plt.show()

## Section 7: JSON Export for LLM Integration

Save summaries as JSON files ready for use by ChatGPT/Gemini.

In [ ]:
import json
from pathlib import Path

# Create output directory
output_dir = Path("physics_summaries")
output_dir.mkdir(exist_ok=True)

# Save summaries
for name, summary in [("colloidal", summary_colloidal),
                      ("cellular", summary_cellular),
                      ("polymer", summary_polymer)]:
    output_file = output_dir / f"{name}_summary.json"
    with open(output_file, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"✓ Saved: {output_file}")

print(f"\nAll summaries saved to {output_dir}/")

## Section 8: LLM-Ready Summary Format

Example prompt for ChatGPT/Gemini that uses the JSON summary.

In [ ]:
def create_llm_prompt(summary: Dict, domain: str = "general") -> str:
    """
    Create a structured prompt for LLM explainer using physics summary.
    
    This prompt can be sent to ChatGPT 4, Gemini Pro, or Claude.
    """
    
    bulk = summary['msd_analysis']['bulk']
    hetero = summary['heterogeneity']
    struct = summary['structure']
    flags = summary['flags_and_anomalies']
    
    prompt = f"""
## Confocal Microscopy Physics Analysis Report

**Domain:** {domain.title()}
**Number of Trajectories:** {summary['metadata']['n_trajectories']}
**Physical Parameters:** dt = {summary['metadata']['dt_physical_units']}, px_to_um = {summary['metadata']['px_to_um_conversion']}

### Key Findings

**1. Diffusion Dynamics (MSD Analysis)**
- Anomaly exponent (α): **{bulk['anomaly_exponent_alpha']:.3f}**
- Interpretation: {bulk['interpretation']}
- Diffusion coefficient: {bulk['diffusion_coefficient']:.3e}

**2. Heterogeneity Metrics**
- Non-Gaussian parameter (α₂) maximum: {hetero['alpha2_max']}
- Peak non-Gaussian lag time: {hetero['alpha2_tau_peak']}
- MSD distribution std: {hetero['msd_distribution_std']}
- MSD distribution mean: {hetero['msd_distribution_mean']}
- Subdiffusion detected: {hetero['subdiffusion_detected']}
- Superdiffusion detected: {hetero['superdiffusion_detected']}
- Dynamic susceptibility (variance proxy): {hetero['dynamic_susceptibility']}

**3. Structure Analysis**
- Clustering detected (S(0) >> 1): {struct['clustering_detected']}
- System density: {struct['density']}
- Number of measured q-vectors: {len(struct['q_values'])}

**4. Anomalies & Flags**
"""
    
    for i, flag in enumerate(flags, 1):
        prompt += f"- {flag}\n"
    
    prompt += f"""
### Domain-Specific Interpretation

"""
    
    if domain.lower() == "colloidal":
        prompt += """In the **colloidal context**, these measurements suggest:
- Brownian motion (α ≈ 1) indicates free diffusion without confinement
- Low heterogeneity (α₂ small) is consistent with hard-sphere dynamics
- Structure factor S(q) reveals packing statistics and possible gel formation

Next steps: Check for gelation, measure interaction potentials, or examine shear response.
"""
    
    elif domain.lower() == "cellular":
        prompt += """In the **cellular context**, these measurements suggest:
- Subdiffusion (α < 1) is typical for nuclear/cytoplasmic diffusion due to crowding
- High heterogeneity (α₂ large) may indicate coexistence of mobile and immobile pools
- Clustering in structure factor could reflect organelle co-localization

Next steps: Segment mobile vs. immobile pools, quantify crowding, or map spatial domains.
"""
    
    elif domain.lower() == "polymer":
        prompt += """In the **polymer context**, these measurements suggest:
- Strong subdiffusion (α << 1) is characteristic of monomer dynamics in entangled chains
- High non-Gaussian parameter reflects local chain rearrangements
- MSD heterogeneity arises from intra-chain friction and excluded volume

Next steps: Characterize chain relaxation, compute entanglement length, or measure viscoelasticity.
"""
    
    else:
        prompt += """General interpretation:
- Compare α to 1.0 to assess whether dynamics are normal (Brownian), subdiffusive, or superdiffusive
- Use heterogeneity metrics to detect dynamic heterogeneity and non-equilibrium effects
- Structure factors reveal spatial correlations and clustering tendencies
"""
    
    prompt += f"""

### JSON Summary (for programmatic use)

```json
{json.dumps(summary, indent=2)}
```
"""
    
    return prompt


# Generate and display example prompts
print("="*70)
print("LLM PROMPT FOR COLLOIDAL SYSTEM")
print("="*70)
prompt_colloidal = create_llm_prompt(summary_colloidal, domain="colloidal")
print(prompt_colloidal[:1000] + "\n... [truncated for display]")

print("\n" + "="*70)
print("LLM PROMPT FOR CELLULAR SYSTEM")
print("="*70)
prompt_cellular = create_llm_prompt(summary_cellular, domain="cellular")
print(prompt_cellular[:1000] + "\n... [truncated for display]")

# Save prompts
with open(output_dir / "colloidal_llm_prompt.txt", 'w') as f:
    f.write(prompt_colloidal)
with open(output_dir / "cellular_llm_prompt.txt", 'w') as f:
    f.write(prompt_cellular)
with open(output_dir / "polymer_llm_prompt.txt", 'w') as f:
    f.write(create_llm_prompt(summary_polymer, domain="polymer"))

print(f"\n✓ All LLM prompts saved to {output_dir}/")

## Section 9: Integration Guide

How to integrate `PhysicsAnalyst` into your existing copilot codebase.

### Step-by-Step Integration

#### 1. Copy to your codebase

```bash
# In your confocal_microscopy-copilot repo
cp physics_analyst_advanced.py src/analysis/physics_analyst_advanced.py
```

#### 2. Update DetectionTrackingWorker to produce Trajectory objects

```python
# src/tracking/detection_tracking_worker.py
from src.analysis.physics_analyst_advanced import Trajectory

class DetectionTrackingWorker:
    def get_trajectories(self) -> List[Trajectory]:
        """Export linked trajectories as Trajectory objects."""
        trajectories = []
        for track_id, positions in self.tracks.items():
            traj = Trajectory(
                particle_id=track_id,
                t=positions['t'],
                x=positions['x'],
                y=positions['y'],
                z=positions['z'],
                confidence=positions.get('confidence'),
                radius=positions.get('radius', 0.5),
                label=positions.get('label', 'particle')
            )
            trajectories.append(traj)
        return trajectories
```

#### 3. Integrate PhysicsAnalyst into your main pipeline

```python
# src/app.py or main.py
from src.analysis.physics_analyst_advanced import PhysicsAnalyst

class MicroscopyAnalysisPipeline:
    def __init__(self, domain: str = "general"):
        self.detector = DetectionTrackingWorker(...)
        self.physics = PhysicsAnalyst(
            dt=0.016,  # 16ms per frame for 60Hz camera
            px_to_um=0.065,  # 65 nm per pixel
            wall_z_threshold=2.0  # 2 um from substrate
        )
        self.domain = domain
    
    def analyze(self):
        # Get trajectories from detector
        trajectories = self.detector.get_trajectories()
        
        # Run physics analysis
        summary = self.physics.summarize_physics(trajectories, domain=self.domain)
        
        # Return JSON for UI/LLM
        return summary
```

#### 4. Connect to your LLM explainer

```python
# src/llm/chat_explainer.py
from src.analysis.physics_analyst_advanced import create_llm_prompt
import openai

class LLMExplainer:
    def explain(self, physics_summary: Dict) -> str:
        """Generate explanation using ChatGPT."""
        
        # Create prompt from physics summary
        prompt = create_llm_prompt(physics_summary, domain="cellular")
        
        # Send to OpenAI
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert in microscopy physics and soft matter dynamics."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=1024
        )
        
        return response['choices'][0]['message']['content']
```

#### 5. Expose in UI

```html
<!-- ui/components/physics_dashboard.html -->
<div id="diagnostics-container">
  <div id="msd-plots"></div>
  <div id="heterogeneity-metrics"></div>
  <div id="structure-analysis"></div>
  <button id="ask-llm">Ask LLM for Explanation</button>
  <div id="llm-explanation"></div>
</div>

<script>
  // Fetch physics summary from backend
  fetch('/api/physics-analysis')
    .then(r => r.json())
    .then(summary => {
      renderMSDPlots(summary.msd_analysis);
      renderHeterogeneityMetrics(summary.heterogeneity);
      renderStructureAnalysis(summary.structure);
      
      // LLM button
      document.getElementById('ask-llm').onclick = () => {
        fetch('/api/explain-physics', {
          method: 'POST',
          body: JSON.stringify(summary)
        })
        .then(r => r.json())
        .then(data => {
          document.getElementById('llm-explanation').innerText = data.explanation;
        });
      };
    });
</script>
```

### Configuration Schema (YAML)

Create a config file for different microscopy setups:

```yaml
# configs/confocal_setups.yaml

setups:
  zeiss_lsm:
    dt_ms: 5.0
    px_to_um: 0.032  # 32 nm pixels
    pinhole_um: 0.5
    wall_threshold_um: 1.5
    domain: "colloidal"
  
  nikon_a1:
    dt_ms: 10.0
    px_to_um: 0.065
    pinhole_um: 1.0
    wall_threshold_um: 2.0
    domain: "general"
  
  live_cell_imaging:
    dt_ms: 100.0  # slower, thermal drift
    px_to_um: 0.1
    pinhole_um: 2.0
    wall_threshold_um: 5.0
    domain: "cellular"
```

Load and use:

```python
import yaml

with open('configs/confocal_setups.yaml', 'r') as f:
    setups = yaml.safe_load(f)['setups']

config = setups['nikon_a1']
analyzer = PhysicsAnalyst(
    dt=config['dt_ms'] / 1000,
    px_to_um=config['px_to_um'],
    wall_z_threshold=config['wall_threshold_um']
)
```

## Section 10: Advanced Topics & Extensions

Future enhancements and research directions.

### A. Time-Dependent Heterogeneity

Track how dynamics evolve in time (e.g., sample aging, aging in glassy systems):

```python
def compute_age_dependent_msd(trajectories, age_windows=5):
    """Compute MSD in temporal blocks to reveal aging."""
    # Divide trajectory into age_windows temporal slices
    # Compute MSD in each slice
    # Plot as heatmap: lag_time vs age
```

### B. Flow-Induced Dynamics

For shear or pressure-driven systems:

```python
def compute_velocity_autocorrelation(trajectories):
    """Compute v(t) · v(0) to detect flow alignment."""
    # For each particle, compute velocity vectors
    # Autocorrelate to detect flow direction
```

### C. Machine Learning for Domain Classification

Train a classifier to automatically assign domain type from physics metrics:

```python
from sklearn.ensemble import RandomForestClassifier

# Training data: (alpha, alpha2_max, dynamic_susceptibility) -> domain label
X = np.array([...])
y = np.array([...])

clf = RandomForestClassifier()
clf.fit(X, y)

def auto_classify_domain(summary):
    features = np.array([[
        summary['msd_analysis']['bulk']['anomaly_exponent_alpha'],
        summary['heterogeneity']['alpha2_max'],
        summary['heterogeneity']['dynamic_susceptibility']
    ]])
    return clf.predict(features)[0]
```

### D. Interactive LLM Feedback Loop

```python
class InteractiveLLMExplainer:
    def __init__(self, model="gpt-4"):
        self.model = model
        self.conversation_history = []
    
    def explain_and_refine(self, physics_summary, follow_up_questions):
        """Multi-turn conversation for deeper insights."""
        # Initial explanation
        prompt = create_llm_prompt(physics_summary)
        initial_response = self.query_llm(prompt)
        self.conversation_history.append(("system", prompt, initial_response))
        
        # Follow-up questions
        for question in follow_up_questions:
            follow_up_prompt = f"""
            Based on the previous analysis, {question}
            
            Physics context:
            {json.dumps(physics_summary, indent=2)}
            """
            response = self.query_llm(follow_up_prompt)
            self.conversation_history.append(("user", follow_up_prompt, response))
        
        return self.conversation_history
```

## Section 11: Best Practices & Troubleshooting

### Common Issues & Solutions

| Issue | Cause | Solution |
|-------|-------|----------|
| α2 always zero | No displacement variance | Check trajectory quality; ensure sufficient time steps |
| S(q) >> 1 everywhere | Random q-vector sampling | Use regular q-grid or average over multiple directions |
| g(r) noisy | Too few particles | Increase ensemble size or smooth with rolling window |
| Wall MSD = Bulk MSD | No particles near wall | Increase wall_z_threshold or check z-calibration |
| Negative MSD values | Numerical error | Check for NaN in positions; use `np.maximum(..., 1e-10)` |
| LLM output generic | Poor context in prompt | Include domain, n_particles, alpha2_max in system prompt |

### Performance Tips

1. **Use stride parameter** to reduce computation:
   ```python
   lag_times, msd = analyzer.compute_msd(traj, stride=2)  # Skip every other lag
   ```

2. **Limit trajectory length** for real-time analysis:
   ```python
   analyzer.compute_msd(traj, max_lag_frames=100)  # Cap at 100 frames
   ```

3. **Cache structure factors** for repeated queries:
   ```python
   cache = {}
   if hash(trajectories) not in cache:
       cache[hash(trajectories)] = analyzer.compute_structure_analysis(trajectories)
   ```

### Validation Checklist

- [ ] MSD increases monotonically with lag time
- [ ] α ≈ 1 for known Brownian particles
- [ ] α2(0) = 0 and α2 increases then decreases
- [ ] S(q→0) > 1 for real systems (compressibility)
- [ ] g(r) ≈ 1 at large r (bulk random)
- [ ] Flags are actionable and domain-specific
- [ ] LLM explanations cite specific values from JSON

## Section 12: References & Further Reading

### Key Physics Papers

1. **MSD and Anomaly Exponents:**
   - Wang et al. (2012). "Anomalous yet Brownian." PNAS 109(20):7641-7646
   - Metzler & Klafter (2000). "The random walk's guide to anomalous diffusion." PhysRep 339(1):1-77

2. **Non-Gaussian Parameter α₂:**
   - Chubynsky & Slater (2005). "Diffusing diffusivity: a model for anomalous diffusion." PRL 113(10):098302

3. **Structure Factors:**
   - Hansen & McDonald (2013). *Theory of Simple Liquids* (Ch. 4)

4. **Confocal Microscopy PSF:**
   - Gibson & Lanni (1992). "Experimental test of an analytical model." JOSA 9(1):154-166

### Tools & Software

- **TrackPy**: Python particle tracking library (http://soft-matter.github.io/trackpy/)
- **DeepTrack 2**: Deep learning for microscopy (http://DeepTrack.itu.dk)
- **Huygens PSF Generator**: Realistic PSF simulations (https://svi.nl)
- **GROMACS/LAMMPS**: MD simulation reference (for validating physics)

### LLM Best Practices

- Keep JSON summaries <4000 tokens to avoid token limits
- Use clear field names ("anomaly_exponent_alpha" > "a")
- Include units in all scalar values
- Provide domain context in system prompt
- Request structured output (bullet points, JSON) from LLM

## Summary: What You've Learned

### ✓ Completed

1. **Implemented PhysicsAnalyst class** with all four diagnostic modules:
   - Bulk MSD with power-law fitting (anomaly exponent α)
   - Near-wall MSD revealing surface-induced hindrance
   - Cage-relative MSD isolating local rearrangements
   - Heterogeneity metrics (α₂, dynamic susceptibility)
   - Structure functions (S(q) and g(r))

2. **Domain-agnostic design:**
   - Works for colloidal, cellular, polymer, and mixed systems
   - Configurable physical parameters (dt, px_to_um, wall_threshold)
   - Domain-aware flag generation

3. **LLM-ready JSON export:**
   - Structured summaries with all metrics
   - Example prompts for ChatGPT/Gemini
   - Integration guide for your existing codebase

4. **Visualizations:**
   - 4-panel diagnostic plots
   - Log-log MSD with power-law overlay
   - Non-Gaussian parameter vs lag time
   - Structure factor and pair correlation functions

### → Next Steps for Your Hackathon

1. **Integrate PhysicsAnalyst into DetectionTrackingWorker**
   - Convert linked trajectories to `Trajectory` objects
   - Test on real confocal data from your dataset

2. **Connect to LLM explainer**
   - Replace dummy explainer with real OpenAI/Gemini client
   - Test multi-turn conversation for follow-up questions

3. **Add UI elements**
   - Render MSD plots and metrics in dashboard
   - Add "Ask LLM" button with streaming response
   - Toggle between domain contexts

4. **Extend with real digital twin**
   - Replace synthetic trajectories with digital-twin-generated data
   - Validate against known physics (Stokes drag, viscoelasticity)

---

**Good luck with your hackathon! 🚀**